In [ ]:
'''GANs employ two neural networks, the Generator and the Discriminator, that are trained simultaneously through adversarial training, with the objective to create new data that is indistinguishable from real data.'''
import torch.nn as nn
def gen_block(in_dim, out_dim):
    return nn.Sequential(
        nn.Linear(in_dim, out_dim),
        nn.BatchNorm1d(out_dim),
        nn.ReLU(inplace=True)
    )

class Generator(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(Generator, self).__init__()
        # Define generator block
        self.generator = ____(
            gen_block(in_dim, 256),
            gen_block(256, 512),
            gen_block(512, 1024),
          	# Add linear layer
            nn.Linear(1024, out_dim),
            # Add activation
            nn.Sigmoid(),
        )

    def forward(self, x):
      	# Pass input through generator
        return self.generator(x)